In [ ]:

from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import os
os.chdir("drive/My Drive/Ben_Boyd_MSc_Project/Data")


In [ ]:
import matplotlib.pyplot as plt
from sklearn import datasets, preprocessing
from IPython.display import clear_output
import numpy as onp
import jax.numpy as np # JAX is supposed to have an API that closely resemble NumPy's
from jax import grad, jit, vmap, value_and_grad
from jax import random
import jax.nn as nn
import copy
%matplotlib inline

from jax.experimental.stax import (Dense, Tanh, Flatten, Relu, LogSoftmax, Softmax, Exp,Sigmoid,Softplus,LeakyRelu)

import matplotlib.pyplot as plt # creating visualizations

from matplotlib import cm
from matplotlib.ticker import LinearLocator

import pickle

from astropy.table import Table

from astropy import table
from astropy.io import fits,ascii,votable
from astropy import units as u 
from astropy import constants as const
from astropy import table

In [ ]:

from jax import nn
from jax.experimental import stax, optimizers

In [ ]:
import jax.numpy as np
import numpy as onp
from jax import nn, ops, random
from jax.experimental import stax

DEFAULT_MIN_BIN_WIDTH = 1e-3
DEFAULT_MIN_BIN_HEIGHT = 1e-3
DEFAULT_MIN_DERIVATIVE = 1e-3


def searchsorted(bin_locations, inputs, eps=1e-6):
    bin_locations = bin_locations.at[..., -1].add(eps)  # bin_locations[..., -1] += eps
    return np.sum(inputs[..., None] >= bin_locations, axis=-1) - 1


def unconstrained_RQS(
    inputs,
    unnormalized_widths,
    unnormalized_heights,
    unnormalized_derivatives,
    inverse=False,
    tail_bound=1.0,
    min_bin_width=DEFAULT_MIN_BIN_WIDTH,
    min_bin_height=DEFAULT_MIN_BIN_HEIGHT,
    min_derivative=DEFAULT_MIN_DERIVATIVE,
):
    #inside_interval_mask = (inputs >= -tail_bound) & (inputs <= tail_bound)
    #outside_interval_mask = ~inside_interval_mask
  

    outputs = np.zeros_like(inputs)
    logabsdet = np.ones_like(inputs)

    unnormalized_derivatives = np.pad(
        unnormalized_derivatives, [(0, 0)] * (len(unnormalized_derivatives.shape) - 1) + [(1, 1)]
    )
    constant = np.log(np.exp(1 - min_derivative) - 1)
    unnormalized_derivatives = unnormalized_derivatives.at[..., 0].set(constant)  # unnormalized_derivatives[..., 0] = constant
    unnormalized_derivatives = unnormalized_derivatives.at[..., -1].set(constant)  # unnormalized_derivatives[..., -1] = constant

        
    logabsdet = np.where(inputs<-tail_bound,0,logabsdet)
    logabsdet = np.where(inputs> tail_bound , 0, logabsdet)
    
    outputs = np.where(logabsdet==0,inputs,outputs)






    #outputs = outputs.at[outside_interval_mask].set(inputs[outside_interval_mask])
    # outputs[outside_interval_mask] = inputs[outside_interval_mask]
    #logabsdet = logabsdet.at[outside_interval_mask].set(0)
     # logabsdet[outside_interval_mask] = 0

    outs, logdets = RQS(
        inputs=inputs,
        unnormalized_widths=unnormalized_widths,
        unnormalized_heights=unnormalized_heights,
        unnormalized_derivatives=unnormalized_derivatives,
        inverse=inverse,
        left=-tail_bound,
        right=tail_bound,
        bottom=-tail_bound,
        top=tail_bound,
        min_bin_width=min_bin_width,
        min_bin_height=min_bin_height,
        min_derivative=min_derivative,
    )

    outputs = np.where(logabsdet==1,outs,outputs)  # outputs[inside_interval_mask] = outs
    logabsdet = np.where(logabsdet==1,logdets,logabsdet) # logabsdet[inside_interval_mask] = logdets

    return outputs, logabsdet


def RQS(
    inputs,
    unnormalized_widths,
    unnormalized_heights,
    unnormalized_derivatives,
    inverse=False,
    left=0.0,
    right=1.0,
    bottom=0.0,
    top=1.0,
    min_bin_width=DEFAULT_MIN_BIN_WIDTH,
    min_bin_height=DEFAULT_MIN_BIN_HEIGHT,
    min_derivative=DEFAULT_MIN_DERIVATIVE,
):


    num_bins = unnormalized_widths.shape[-1]

    if min_bin_width * num_bins > 1.0:
        raise ValueError("Minimal bin width too large for the number of bins")
    if min_bin_height * num_bins > 1.0:
        raise ValueError("Minimal bin height too large for the number of bins")

    widths = nn.softmax(unnormalized_widths, axis=-1)
    widths = min_bin_width + (1 - min_bin_width * num_bins) * widths
    cumwidths = np.cumsum(widths, axis=-1)
    cumwidths = np.pad(
        cumwidths, [(0, 0)] * (len(cumwidths.shape) - 1) + [(1, 0)], mode="constant", constant_values=0.0
    )
    cumwidths = (right - left) * cumwidths + left
    cumwidths = cumwidths.at[..., 0].set(left)  # cumwidths[..., 0] = left
    cumwidths = cumwidths.at[..., -1].set(right)  # cumwidths[..., -1] = right
    widths = cumwidths[..., 1:] - cumwidths[..., :-1]

    derivatives = min_derivative + nn.softplus(unnormalized_derivatives)

    heights = nn.softmax(unnormalized_heights, axis=-1)
    heights = min_bin_height + (1 - min_bin_height * num_bins) * heights
    cumheights = np.cumsum(heights, axis=-1)
    cumheights = np.pad(
        cumheights, [(0, 0)] * (len(cumheights.shape) - 1) + [(1, 0)], mode="constant", constant_values=0.0
    )
    cumheights = (top - bottom) * cumheights + bottom
    cumheights = cumheights.at[..., 0].set(bottom)  # cumheights[..., 0] = bottom
    cumheights = cumheights.at[..., -1].set(top)  # cumheights[..., -1] = top
    heights = cumheights[..., 1:] - cumheights[..., :-1]

    if inverse:
        bin_idx = searchsorted(cumheights, inputs)[..., None]
    else:
        bin_idx = searchsorted(cumwidths, inputs)[..., None]

    input_cumwidths = np.take_along_axis(cumwidths, bin_idx, -1)[..., 0]  # cumwidths.gather(-1, bin_idx)[..., 0]
    input_bin_widths = np.take_along_axis(widths, bin_idx, -1)[..., 0]  # widths.gather(-1, bin_idx)[..., 0]

    input_cumheights = np.take_along_axis(cumheights, bin_idx, -1)[..., 0]  # cumheights.gather(-1, bin_idx)[..., 0]
    delta = heights / widths
    input_delta = np.take_along_axis(delta, bin_idx, -1)[..., 0]  # delta.gather(-1, bin_idx)[..., 0]

    input_derivatives = np.take_along_axis(derivatives, bin_idx, -1)[..., 0]  # derivatives.gather(-1, bin_idx)[..., 0]
    input_derivatives_plus_one = np.take_along_axis(
        derivatives[..., 1:], bin_idx, -1
    )  # derivatives[..., 1:].gather(-1, bin_idx)
    input_derivatives_plus_one = input_derivatives_plus_one[..., 0]

    input_heights = np.take_along_axis(heights, bin_idx, -1)[..., 0]  # heights.gather(-1, bin_idx)[..., 0]

    if inverse:
        a = (inputs - input_cumheights) * (
            input_derivatives + input_derivatives_plus_one - 2 * input_delta
        ) + input_heights * (input_delta - input_derivatives)
        b = input_heights * input_derivatives - (inputs - input_cumheights) * (
            input_derivatives + input_derivatives_plus_one - 2 * input_delta
        )
        c = -input_delta * (inputs - input_cumheights)

        discriminant = np.square(b) - 4 * a * c

        root = (2 * c) / (-b - np.sqrt(discriminant))
        outputs = root * input_bin_widths + input_cumwidths

        theta_one_minus_theta = root * (1 - root)
        denominator = input_delta + (
            (input_derivatives + input_derivatives_plus_one - 2 * input_delta) * theta_one_minus_theta
        )
        derivative_numerator = np.square(input_delta) * (
            input_derivatives_plus_one * np.square(root)
            + 2 * input_delta * theta_one_minus_theta
            + input_derivatives * np.square(1 - root)
        )
        logabsdet = np.log(derivative_numerator) - 2 * np.log(denominator)
        return outputs, -logabsdet
    else:
        theta = (inputs - input_cumwidths) / input_bin_widths
        theta_one_minus_theta = theta * (1 - theta)

        numerator = input_heights * (input_delta * np.square(theta) + input_derivatives * theta_one_minus_theta)
        denominator = input_delta + (
            (input_derivatives + input_derivatives_plus_one - 2 * input_delta) * theta_one_minus_theta
        )
        outputs = input_cumheights + numerator / denominator

        derivative_numerator = np.square(input_delta) * (
            input_derivatives_plus_one * np.square(theta)
            + 2 * input_delta * theta_one_minus_theta
            + input_derivatives * np.square(1 - theta)
        )
        logabsdet = np.log(derivative_numerator) - 2 * np.log(denominator)
        return outputs, logabsdet





def NeuralSpline1D(network,K=5, B=3, hidden_dim=64):
    def init_fun(rng, input_dim, **kwargs):
        params, apply_fun = network(rng,input_dim,3*K-1,hidden_dim)

        def direct_fun(params, x):
            log_det = np.zeros(x.shape[0])
            out = apply_fun(params,x[:,1:]).reshape(-1, 1,3 * K - 1)
            W, H, D = np.array_split(out, 3, axis=2)
            W, H = nn.softmax(W, axis=2), nn.softmax(H, axis=2)
            W, H = 2 * B * W, 2 * B * H
            D = nn.softplus(D)
            out, ld = unconstrained_RQS( x[:,:1], W, H, D, inverse=False, tail_bound=B)
            log_det += np.sum(ld, axis=1)
            return np.concatenate([out,x[:,1:]], axis=1), log_det.reshape((x.shape[0],))

        def inverse_fun(params, z):
            log_det = np.zeros(z.shape[0])
            out = apply_fun(params,z[:,1:]).reshape(-1, 1,3 * K - 1)
            W, H, D = np.array_split(out, 3, axis=2)
            W, H = nn.softmax(W, axis=2), nn.softmax(H, axis=2)
            W, H = 2 * B * W, 2 * B * H
            D = nn.softplus(D)
            out, ld = unconstrained_RQS( z[:,:1], W, H, D, inverse=True, tail_bound=B)
            log_det += np.sum(ld, axis=1)
            return np.concatenate([out,z[:,1:]], axis=1), log_det.reshape((z.shape[0],))

        return params, direct_fun, inverse_fun

    return init_fun


In [ ]:


def Serial(*init_funs):
    """
    Args:
        *init_funs: Multiple bijections in sequence

    Returns:
        An ``init_fun`` mapping ``(rng, input_dim)`` to a ``(params, direct_fun, inverse_fun)`` triplet.

    Examples:
        >>> num_examples, input_dim, tol = 20, 3, 1e-4
        >>> layer_rng, input_rng = random.split(random.PRNGKey(0))
        >>> inputs = random.uniform(input_rng, (num_examples, input_dim))
        >>> init_fun = Serial(Shuffle(), Shuffle())
        >>> params, direct_fun, inverse_fun = init_fun(layer_rng, input_dim)
        >>> mapped_inputs = direct_fun(params, inputs)[0]
        >>> reconstructed_inputs = inverse_fun(params, mapped_inputs)[0]
        >>> np.allclose(inputs, reconstructed_inputs).item()
        True
    """

    def init_fun(rng, conditional_dim, **kwargs):
        all_params, direct_funs, inverse_funs = [], [], []
        for init_fun in init_funs:
            rng, layer_rng = random.split(rng)
            param, direct_fun, inverse_fun = init_fun(layer_rng,conditional_dim)
            all_params.append(param)
            direct_funs.append(direct_fun)
            inverse_funs.append(inverse_fun)


        def feed_forward(params, apply_funs, inputs):
            log_det_jacobians = np.zeros(inputs.shape[0])
            
            for apply_fun, param in zip(apply_funs, params):

                inputs, log_det_jacobian = apply_fun(param,inputs,**kwargs)
                log_det_jacobians += log_det_jacobian  

            return inputs, log_det_jacobians

            
        def direct_fun(params, inputs, **kwargs):
            return feed_forward(params, direct_funs, inputs)

        def inverse_fun(params, inputs, **kwargs):
            return feed_forward(reversed(params), reversed(inverse_funs), inputs)

        return all_params, direct_fun, inverse_fun

    return init_fun


In [ ]:
from jax.scipy.stats import norm, multivariate_normal,uniform


def Normal():
    """
    Returns:
        A function mapping ``(rng, input_dim)`` to a ``(params, log_pdf, sample)`` triplet.
    """

    def init_fun(rng, input_dim):
        def log_pdf(params, inputs):
            return norm.logpdf(inputs,loc=0,scale=1).sum(1)

        def sample(rng, params, num_samples=1):
            return random.normal(rng, (num_samples, input_dim))*1

        return (), log_pdf, sample

    return init_fun

def Flow(transformation, prior=Normal()):
    """
    Args:
        transformation: a function mapping ``(rng, input_dim)`` to a
            ``(params, direct_fun, inverse_fun)`` triplet
        prior: a function mapping ``(rng, input_dim)`` to a
            ``(params, log_pdf, sample)`` triplet

    Returns:
        A function mapping ``(rng, input_dim)`` to a ``(params, log_pdf, sample)`` triplet.

    Examples:
        >>> import flows
        >>> input_dim, rng = 3, random.PRNGKey(0)
        >>> transformation = flows.Serial(
        ...     flows.Reverse(),
        ...     flows.Reverse()
        ... )
        >>> init_fun = flows.Flow(transformation, Normal())
        >>> params, log_pdf, sample = init_fun(rng, input_dim)
    """

    def init_fun(rng,conditional_dim):
        transformation_rng, prior_rng = random.split(rng)
        params, direct_fun, inverse_fun = transformation(transformation_rng,conditional_dim)
        prior_params, prior_log_pdf, prior_sample = prior(prior_rng, 1)

        def log_pdf(params, inputs):
            u, log_det = direct_fun(params, inputs)
            log_probs = prior_log_pdf(prior_params, u[:,:1])
            return log_probs + log_det
        def sample(rng, params,conditionals, num_samples=1):
            prior_samples = prior_sample(rng, prior_params, num_samples)
            x=inverse_fun(params, np.column_stack((prior_samples,conditionals)))[0]
            return x

        def log_pdf_grid(params,conditionals,search_range=1,searches=6000):
            gals=conditionals.shape[0]
            z_grid_1D=np.linspace(0,search_range,searches).reshape(1,searches)
            z_grid=np.repeat(z_grid_1D,gals,axis=0)
            prob_grid=np.zeros((gals,searches))
            for i in range(searches):

              prob_grid=prob_grid.at[:,i].set(log_pdf(params,np.column_stack((z_grid[:,i].reshape(gals,1),conditionals))))
              print(i)


            pred=z_grid[np.arange(len(z_grid)),np.argmax(prob_grid,axis=1)]

            return z_grid_1D[0,:],prob_grid,pred

        def sample_pred(rng,params,conditionals,samples=1000):
          gals=conditionals.shape[0]
          z_samples=np.zeros((gals,samples))
          prob_grid=np.zeros((gals,samples))
          for i in range(samples):
            print(i)
            rng,sample_rng  = random.split(rng)
            z_samples=z_samples.at[:,i].set(sample(sample_rng,params,conditionals,gals)[:,0])
            prob_grid=prob_grid.at[:,i].set(log_pdf(params,np.column_stack((z_samples[:,i].reshape(gals,1),conditionals))))

          return z_samples[np.arange(len(z_samples)),np.argmax(prob_grid,axis=1)]



        return params, log_pdf,log_pdf_grid, sample,sample_pred
    return init_fun

def network(rng,conditional_dim,out_dim, hidden_dim):
    init_fun,apply_fun=stax.serial(stax.Dense(hidden_dim), Relu, stax.Dense(1024), Relu, stax.Dense(hidden_dim), Relu, stax.Dense(out_dim),)
    _, params = init_fun(rng, (conditional_dim,))
    return params,apply_fun

In [ ]:

n_input=54

rng, flow_rng = random.split(random.PRNGKey(0))

init_fun = Flow(Serial(*(NeuralSpline1D(network,K=5,B=3,hidden_dim=512),)*3),prior=Normal())
params, log_pdf,log_pdf_grid,sample,sample_pred = init_fun(flow_rng,n_input )
print(params)

In [ ]:
!pip install optax
import optax
constant_scheduler = optax.constant_schedule(0.001)

opt_init, opt_update, get_params = optimizers.adam(constant_scheduler)
opt_state = opt_init(params)

In [ ]:
from tqdm.notebook import trange
import itertools
import numpy.random as npr

In [ ]:
@jit
def loss_fn(params, inputs):
    return -log_pdf(params, inputs).mean()

@jit
def step(i, opt_state, inputs):
    params = get_params(opt_state)
    loss,gradients = value_and_grad(loss_fn)(params,inputs)
    return loss, opt_update(i, gradients, opt_state)

In [ ]:

n_samples =  15000000 # just use train everything in a batch
beta=True
small_range=True
magnitudes=False
a=0
if beta:
  a+=5

x_data=onp.zeros((n_samples,54))
y_data=onp.zeros(n_samples)

cats=int(n_samples/1e6)

def mag_err(flux_err,flux):
    
    return 1.09*flux_err/flux

def mag(flux):
    x=-2.5*onp.log10(flux) + 23.9 
    x[onp.isnan(x)]=0
    return x 


for c in range(cats):
    cat = table.Table.read('sim/mil'+str(c+1)+'_noisy_gal.fits',format='fits',hdu=1)
    keys=cat.keys()
    count=0
    y_data[(c)*1000000:(c+1)*1000000]=cat['redshift']
    for key_name in keys:

        if key_name[len(key_name)-9:]=='BETA_FLUX':

                
            filt=key_name[:len(key_name)-10+a]

            if magnitudes:

              x_data[(c)*1000000:(c+1)*1000000,count]=mag(cat[filt+'_FLUX'])
              x_data[(c)*1000000:(c+1)*1000000,count+1]=mag_err(cat[filt+'_FLUXERR'],cat[filt+'_FLUX'])
            else:
              x_data[(c)*1000000:(c+1)*1000000,count]=cat[filt+'_FLUX']
              x_data[(c)*1000000:(c+1)*1000000,count+1]=cat[filt+'_FLUXERR']
            
            count+=2
            
            
        
    print(c)


x_data = np.array(x_data.reshape(n_samples, 54))

if n_input!=54:
  x_data=x_data[:,:n_input]

if small_range:
  ids=np.logical_and(y_data>=0,y_data<=2)
  y_data=y_data[ids]
  x_data=x_data[ids,:]

print(x_data)

y_data = np.array(y_data.reshape(len(y_data), 1))

cat=0
def norm_fit_and_scale(X):
  mean= np.mean(X,axis=0)
  std = np.std(X,axis=0)
  X_std = (X - mean) / std
  return X_std,mean,std

def minmax_fit_and_scale(X):
  max= np.max(X,axis=0)
  min = np.min(X,axis=0)
  X_std = (X - min) / (max-min)
  return X_std,min,max

def minmax_scale(X,min,max):
  return (X - min) / (max - min)

def minmax_unscale(X,min,max):
  return X * (max - min) + min

X,mean,std=minmax_fit_and_scale(np.column_stack((y_data,x_data)))



In [ ]:
itercount = itertools.count()
batch_size=100000
test_name='NSF_low_redshift'
import logging
itercount = itertools.count()
restore=False
saveopt=True
if restore:
  opt_params = pickle.load(open('NSF/'+test_name+'_opt.pkl', "rb"))
  opt_state = optimizers.pack_optimizer_state(opt_params)

for epoch in range(0,10000,1):
    
    permute_rng, rng = random.split(rng)
    #X = random.permutation(permute_rng, X)
    permute_rng, rng = random.split(rng)
    #clear_output(wait=True)
    avg_loss=0
    for batch_index in range(0, len(X), batch_size):
        #loss,opt_state = step(next(itercount), opt_state, X[batch_index:batch_index+batch_size,:])
        loss,opt_state = step(next(itercount), opt_state, X[batch_index:batch_index+batch_size,:])
        params = get_params(opt_state)
        avg_loss+=loss

    if (epoch+1)%100==0:
        if not np.isnan(loss):
          saved_params=copy.deepcopy(params)
          np.save('NSF/'+test_name+'_more_params.npy',saved_params,allow_pickle=True)
          if saveopt:
            trained_params = optimizers.unpack_optimizer_state(opt_state)
            pickle.dump(trained_params, open('NSF/'+test_name+'_opt.pkl', "wb"))




    print('epoch: ',epoch,' loss: ', avg_loss/(int(len(X)/batch_size)))
